(python-libraries)=
# Python Libraries
The following libraries are available and recommended for use by Cal-ITP data analysts.

## Table of Contents
1. [calitp](#calitp)
1. [siuba](#siuba)
<br> - [Basic Query](#basic-query)
<br> - [Collect Query Results](#collect-query-results)
<br> - [Show Query SQL](#show-query-sql)
<br> - [More siuba Resources](more-siuba-resources)
1. [shared utils](shared-utils)
1. [pandas](pandas-resources)
1. [Add New Packages](#add-new-packages)

(calitp)=
## calitp
`calitp` is an internal library of utility functions used to access our warehouse data.

### import tbl

Most notably, you can include `import tbl` at the top of your notebook to import a table from the warehouse in the form of a `tbl`:

```python
from calitp.tables import tbl
```

Example:

In [1]:
from calitp.tables import tbl

tbl.views.gtfs_schedule_fact_daily_feed_routes()

,feed_key,route_key,date,calitp_extracted_at,calitp_deleted_at
0,3973736280699853333,-3751807270254046912,2022-03-01,2022-02-17,2022-03-28
1,-1900869789117407976,2898692629114136014,2021-08-26,2021-06-10,2022-02-15
2,4265329404163977973,-7926705621079462224,2021-08-26,2021-07-02,2021-12-22
3,-7791947413599409076,7326381090358817724,2022-03-01,2022-02-04,2022-03-11
4,4801776436342873542,7314203506288307709,2022-01-17,2022-01-17,2022-01-18


### query_sql

`query_sql` is another useful function to use inside of JupyterHub notebooks to turn a SQL query into a pandas DataFrame.

As an example, in a notebook:

In [2]:
from calitp import query_sql

In [3]:
df_dim_feeds = query_sql("""
SELECT
    *
FROM `views.gtfs_schedule_dim_feeds`
LIMIT 10""", as_df=True)

In [4]:
df_dim_feeds.head()

,feed_key,calitp_itp_id,calitp_url_number,calitp_agency_name,raw_gtfs_schedule_url,calitp_id_in_latest,calitp_feed_id,calitp_feed_name,feed_publisher_name,feed_publisher_url,feed_lang,default_lang,feed_version,feed_contact_email,feed_contact_url,feed_start_date,feed_end_date,is_composite_feed,calitp_extracted_at,calitp_deleted_at
0,-5411475776957284538,343,0,Merced The Bus,https://thebuslive.com/gtfs,True,343__0,Merced The Bus (0),GMV Syncromatics,https://gmvsyncromatics.com,en-US,en-US,4/6/2022,support@gmvsync.com,https://gmvsyncromatics.com/contact,2022-04-05,None,False,2022-04-06,2099-01-01
1,3260374022872529920,257,1,PresidioGo Shuttle,https://presidiobus.com/gtfs,False,257__1,PresidioGo Shuttle (1),GMV Syncromatics,https://gmvsyncromatics.com,en-US,en-US,11/1/2020,support@gmvsync.com,https://gmvsyncromatics.com/contact,2020-10-31,None,False,2021-05-13,2021-06-14
2,-3703239707065078282,257,0,PresidiGo,https://presidiobus.com/gtfs,True,257__0,PresidiGo (0),GMV Syncromatics,https://gmvsyncromatics.com,en-US,en-US,10/28/2021,support@gmvsync.com,https://gmvsyncromatics.com/contact,2021-10-27,None,False,2021-10-28,2099-01-01
3,-3677138142181972672,257,0,PresidiGo,https://presidiobus.com/gtfs,True,257__0,PresidiGo (0),GMV Syncromatics,https://gmvsyncromatics.com,en-US,en-US,11/1/2020,support@gmvsync.com,https://gmvsyncromatics.com/contact,2020-10-31,None,False,2021-05-13,2021-10-28
4,7412050518366669464,483,0,Tulare InterModal Express,https://tularetransit.com/gtfs,True,483__0,Tulare InterModal Express (0),GMV Syncromatics,https://gmvsyncromatics.com,en-US,en-US,back to original schedule,support@gmvsync.com,https://gmvsyncromatics.com/contact,2020-07-28,None,False,2022-03-01,2099-01-01


(siuba)=
## siuba
`siuba` is a tool that allows the same analysis code to run on a pandas DataFrame,
as well as generate SQL for different databases.
It supports most [pandas Series methods](https://pandas.pydata.org/pandas-docs/stable/reference/series.html) analysts use. See the [siuba docs](https://siuba.readthedocs.io) for more information.

The examples below go through the basics of using siuba, collecting a database query to a local DataFrame,
and showing SQL test queries that siuba code generates.

### Basic query

In [5]:
from myst_nb import glue
from calitp.tables import tbl
from siuba import _, filter, count, collect, show_query

# query lastest validation notices, then filter for a single gtfs feed,
# and then count how often each code occurs
(tbl.views.gtfs_schedule_dim_feeds()
    >> filter(_.calitp_itp_id == 10, _.calitp_url_number==0)
    >> count(_.feed_key)
)

,feed_key,n
0,1619926170103152824,1
1,2570854701378106641,1
2,-5013919702465349414,1
3,4353298498747921001,1
4,-8003123425407053862,1


### Collect query results
Note that siuba by default prints out a preview of the SQL query results.
In order to fetch the results of the query as a pandas DataFrame, run `collect()`.

In [6]:
tbl_agency_names = tbl.views.gtfs_schedule_dim_feeds() >> collect()

# Use pandas .head() method to show first 5 rows of data
tbl_agency_names.head()


,feed_key,calitp_itp_id,calitp_url_number,calitp_agency_name,raw_gtfs_schedule_url,calitp_id_in_latest,calitp_feed_id,calitp_feed_name,feed_publisher_name,feed_publisher_url,feed_lang,default_lang,feed_version,feed_contact_email,feed_contact_url,feed_start_date,feed_end_date,is_composite_feed,calitp_extracted_at,calitp_deleted_at
0,-5411475776957284538,343,0,Merced The Bus,https://thebuslive.com/gtfs,True,343__0,Merced The Bus (0),GMV Syncromatics,https://gmvsyncromatics.com,en-US,en-US,4/6/2022,support@gmvsync.com,https://gmvsyncromatics.com/contact,2022-04-05,None,False,2022-04-06,2099-01-01
1,3260374022872529920,257,1,PresidioGo Shuttle,https://presidiobus.com/gtfs,False,257__1,PresidioGo Shuttle (1),GMV Syncromatics,https://gmvsyncromatics.com,en-US,en-US,11/1/2020,support@gmvsync.com,https://gmvsyncromatics.com/contact,2020-10-31,None,False,2021-05-13,2021-06-14
2,-3703239707065078282,257,0,PresidiGo,https://presidiobus.com/gtfs,True,257__0,PresidiGo (0),GMV Syncromatics,https://gmvsyncromatics.com,en-US,en-US,10/28/2021,support@gmvsync.com,https://gmvsyncromatics.com/contact,2021-10-27,None,False,2021-10-28,2099-01-01
3,-3677138142181972672,257,0,PresidiGo,https://presidiobus.com/gtfs,True,257__0,PresidiGo (0),GMV Syncromatics,https://gmvsyncromatics.com,en-US,en-US,11/1/2020,support@gmvsync.com,https://gmvsyncromatics.com/contact,2020-10-31,None,False,2021-05-13,2021-10-28
4,7412050518366669464,483,0,Tulare InterModal Express,https://tularetransit.com/gtfs,True,483__0,Tulare InterModal Express (0),GMV Syncromatics,https://gmvsyncromatics.com,en-US,en-US,back to original schedule,support@gmvsync.com,https://gmvsyncromatics.com/contact,2020-07-28,None,False,2022-03-01,2099-01-01


### Show query SQL

While `collect()` fetches query results, `show_query()` prints out the SQL code that siuba generates.

In [7]:
(tbl.views.gtfs_schedule_dim_feeds()
  >> filter(_.calitp_agency_name.str.contains("Metro"))
  >> show_query(simplify=True)
)


SELECT `anon_1`.`feed_key`, `anon_1`.`calitp_itp_id`, `anon_1`.`calitp_url_number`, `anon_1`.`calitp_agency_name`, `anon_1`.`raw_gtfs_schedule_url`, `anon_1`.`calitp_id_in_latest`, `anon_1`.`calitp_feed_id`, `anon_1`.`calitp_feed_name`, `anon_1`.`feed_publisher_name`, `anon_1`.`feed_publisher_url`, `anon_1`.`feed_lang`, `anon_1`.`default_lang`, `anon_1`.`feed_version`, `anon_1`.`feed_contact_email`, `anon_1`.`feed_contact_url`, `anon_1`.`feed_start_date`, `anon_1`.`feed_end_date`, `anon_1`.`is_composite_feed`, `anon_1`.`calitp_extracted_at`, `anon_1`.`calitp_deleted_at` 
FROM (SELECT feed_key, calitp_itp_id, calitp_url_number, calitp_agency_name, raw_gtfs_schedule_url, calitp_id_in_latest, calitp_feed_id, calitp_feed_name, feed_publisher_name, feed_publisher_url, feed_lang, default_lang, feed_version, feed_contact_email, feed_contact_url, feed_start_date, feed_end_date, is_composite_feed, calitp_extracted_at, calitp_deleted_at 
FROM `views.gtfs_schedule_dim_feeds`) AS `anon_1` 
WHERE r

,feed_key,calitp_itp_id,calitp_url_number,calitp_agency_name,raw_gtfs_schedule_url,calitp_id_in_latest,calitp_feed_id,calitp_feed_name,feed_publisher_name,feed_publisher_url,feed_lang,default_lang,feed_version,feed_contact_email,feed_contact_url,feed_start_date,feed_end_date,is_composite_feed,calitp_extracted_at,calitp_deleted_at
0,-6941377690184990428,296,0,Santa Cruz Metropolitan Transit District,http://scmtd.com/google_transit/google_transit...,True,296__0,Santa Cruz Metropolitan Transit District (0),Santa Cruz Metro,http://www.scmtd.com,en,None,None,gtfsupdates@scmtd.com,https://www.scmtd.com/en/riders-guide/about-go...,None,None,False,2021-12-10,2099-01-01
1,-4045042305146455543,296,0,Santa Cruz Metropolitan Transit District,http://scmtd.com/google_transit/google_transit...,True,296__0,Santa Cruz Metropolitan Transit District (0),Santa Cruz Metro,http://www.scmtd.com,en,None,None,gtfsupdates@scmtd.com,https://scmtd.com/en/riders-guide/google-trans...,None,None,False,2021-09-07,2021-12-10
2,4164903538710312962,182,0,Metro,https://gitlab.com/LACMTA/gtfs_bus/raw/master/...,True,182__0,Metro (0),Los Angeles County Metropolitan Transportation...,https://www.metro.net,en,None,None,None,https://developer.metro.net,None,None,False,2021-12-18,2099-01-01
3,-3313272383092544529,182,0,Metro,https://gitlab.com/LACMTA/gtfs_bus/raw/master/...,True,182__0,Metro (0),Los Angeles County Metropolitan Transportation...,https://www.metro.net,en,None,None,None,http://developer.metro.net,None,None,False,2021-04-15,2021-12-18
4,1528025220739626551,182,1,Metro,https://gitlab.com/LACMTA/gtfs_rail/raw/master...,True,182__1,Metro (1),Los Angeles County Metropolitan Transportation...,https://www.metro.net,en,None,None,None,http://developer.metro.net,None,None,False,2021-04-15,2099-01-01


Note that here the pandas Series method `str.contains` corresponds to `regexp_contains` in Google BigQuery.

(more-siuba-resources)=
### More siuba Resources:
* [siuba docs](https://siuba.readthedocs.io)
* ['Tidy Tuesday' live analyses with siuba](https://www.youtube.com/playlist?list=PLiQdjX20rXMHc43KqsdIowHI3ouFnP_Sf)

(shared-utils)=
## shared utils
A set of shared utility functions can also be installed, similarly to any Python library. The [shared_utils](https://github.com/cal-itp/data-analyses/tree/main/_shared_utils) are stored here. Generalized functions for analysis are added as collaborative work evolves so we aren't constantly reinventing the wheel.

```python
# In terminal:
cd data-analyses/_shared_utils

# Use the make command to run through conda install and pip install
make setup_env

# In notebook:
import shared_utils

shared_utils.geography_utils.WGS84

# Note: you may need to select Kernel -> Restart Kernel from the top menu
# after make setup_env in order to successfully import shared_utils
```

See [data-analyses/example_reports](https://github.com/cal-itp/data-analyses/tree/main/example_report) for examples on how to use `shared_utils` for general functions, charts, and maps.

(pandas-resources)=
## pandas
The library pandas is very commonly used in data analysis, and the external resources below provide a brief overview of it's use.

* [Cheat Sheet - pandas](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)

## Add New Packages

While most Python packages an analyst uses come in JupyterHub, there may be additional packages you'll want to use in your analysis.

* Install [shared utility functions](#shared-utils)
* Change directory into the project task's subfolder and add `requirements.txt` and/or `conda-requirements.txt`
* Run `pip install -r requirements.txt` and/or `conda install --yes -c conda-forge --file conda-requirements.txt`